In [1]:
%pip install requests pandas geopandas shapely

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached pandas-2.2.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached geopandas-0.14.4-py3-none-any.whl.metadata (1.5 kB)
  Using cached shapely-2.0.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (7.0 kB)
  Using cached charset_normalizer-3.3.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached fiona-1.9.6-cp39-cp39-macosx_11_0_arm64.whl.metadata (50 kB)
  Using cached pyproj-3.6.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (31 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Usin

In [ ]:
from datetime import datetime
from typing import Optional
from abc import ABC, abstractmethod

import requests
import logging
import re

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

In [4]:
DATA_COLLECTION = "SENTINEL-2"
DATE_FORMAT = "%Y-%m-%dT00:00:00.000Z"
COORDINATES_REGEX = "^[-+]?([1-8]?\d(\.\d+)?|90(\.0+)?),\s*[-+]?(180(\.0+)?|((1[0-7]\d)|([1-9]?\d))(\.\d+)?)$"
AUTH_URL  = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
PRODUCTS_URL = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"
DOWNLOAD_URL = "https://download.dataspace.copernicus.eu/odata/v1/Products"

In [5]:
# TODO: Move username and password into an .env file and use python env or os.environ to access the values
copernicus_username = "21ssds415008@msruas.ac.in"
copernicus_password = "Myresearch@2021"

In [6]:
class Coordinates(ABC):
    @abstractmethod
    def stringify(self) -> str:
        pass

class Point(Coordinates):
    longitude: str
    latitude: str

    def __init__(self, longitude: str, latitude: str) -> None:
        is_coordinate = re.match(COORDINATES_REGEX, f"{latitude}, {longitude}")

        if not is_coordinate:
            raise Exception(
                "The given coordinate string does not match the type of a coordinate"
            )

        self.longitude = longitude
        self.latitude = latitude

    def __str__(self) -> str:
        return f"{self.latitude} {self.longitude}"

    def stringify(self) -> str:
        return f"POINT({self})"

class Polygon(Coordinates):
    def __init__(self, coordinates: list[Point]) -> None:
        if len(coordinates) < 4:
            raise ValueError("There must be atleast 4 coordinates")

        if coordinates[-1].stringify() != coordinates[0].stringify():
            raise ValueError("There starting and ending coordinates should be the same")

        self.coordinates = coordinates

    def __str__(self) -> str:
        coordinates_list = [str(coordinate) for coordinate in self.coordinates]
        return ",".join(coordinates_list)

    def stringify(self):
        return f"POLYGON(({self}))"


In [8]:
def get_access_token(username: str, password: str) -> Optional[str]:
    logging.info(f"Username: {username}, Password: {password}")
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }

    response = request.post(AUTH_URL, data=data)
    if response.status_code != 200:
        logging.error(f"Could not get access token, request returned status code {response.status_code}")
        return None
    
    json_response = response.json()
    return json_response.get("access_token")

In [ ]:
def get_report(coordinates: Coordinates, start_date: datetime, end_date: datetime, top: int = 1000, count: bool = True) -> Optional[dict]:
    filters = [
        f"Collection/Name eq '{DATA_COLLECTION}'",
        f"OData.CSC.Intersects(area=geography'SRID=4326;{coordinates.stringify()}')",
        f"ContentDate/Start gt {start_date.strftime(DATE_FORMAT)}",
        f"ContentDate/Start lt {end_date.strftime(DATE_FORMAT)}",
    ]

    url = PRODUCTS_URL + f"?$filter={' and '.join(filters)}&$count={"True" if count else "False"}&$top={top}"
    logging.info(f"Sending a get request to {url}")

    response = requests.get(url)
    if response.status_code != 200:
        logging.error(f"Could not get report, request returned status code {response.status_code}")
        return None
    
    json_data = response.json()
    return json_data.get("value")

In [ ]:
def process_data(data: dict):
    dataframes = pd.DataFrame.from_dict(data)
    if dataframes.shape[0] > 0:
        dataframes['geometry'] = dataframes["GeoFootprint"].apply(shape)
        geo_dataframes = gpd.GeoDataFrame(dataframes).set_geometry("geometry") # Convert PD to GPD
        geo_dataframes = geo_dataframes[~geo_dataframes["Name"].str.contians("L1C")]
        logging.info(f"Total L2A tiles found {len(geo_dataframes)}")
        geo_dataframes["identifier"] = geo_dataframes["Name"].str.split(".").str[0]
        
        return geo_dataframes

In [ ]:
def download_image(token: str, uuid: str) -> None:
    session = requests.Session()
    headers = {"Authorization": f"Bearer {token}"}
    session.headers.update(headers)
    url = DOWNLOAD_URL + f"({uuid})/$value"
    response = session.get(url, headers=headers, stream=True)
    if response.status_code != 200:
        logging.error(f"Unable to download the image, request returned with {response.status_code}.")
        return
    with open(f"{uuid}.zip", "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)
            logging.info(f"Finished Downloading the image")

In [ ]:
if __name__ == "__main__":

    point = Point(longitude="57.692102513344906", latitude="-20.4745116407249")
    polygon = Polygon(
        coordinates=[
            Point("57.692102513344906", "-20.35226228118077"),
            Point("57.692102513344906", "-20.4745116407249"),
            Point("57.77699025382094", "-20.4745116407249"),
            Point("57.77699025382094", "-20.35226228118077"),
            Point("57.692102513344906", "-20.35226228118077"),
        ]
    )
    start_date = datetime(2020, 9, 10)
    end_date = datetime(2020, 9, 20)


    report = get_report(polygon, start_date, end_date)
    if report:
        data = process_data(report)
        if data:
            token = get_access_token(copernicus_username, copernicus_password)
            for index, feature in enumerate(data.iterfeatures()):
                download_image(token, feature['properties']['Id'])

In [1]:
%pip install flask

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.7 MB/s eta 0:00:00a 0:00:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 1.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 1.3 MB/s eta 0:00:00 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [2]:
from flask import Flask, request

In [3]:
app = Flask(__name__)

@app.route("/images")
def get_images():
    latitude = request.args.get('latitude')
    longitude = request.args.get('longitude')
    from_date = request.args.get("from")
    to_date = request.args.get("to")

    if latitude and longitude and from_date and to_date:
        images = []
        report = get_report(Point(latitude=latitude, longitude=longitude), datetime(from_date), datetime(to_date))
        if (report):
            data = process_data(report)
            if data:
                token = get_access_token(copernicus_username, copernicus_password)
                for feature in data.iterfeatures():
                    # TODO: This is going to take a long time due to some images being more than 100MB to download so insted show a wait time
                    images.append(download_image(token, feature["property"]["Id"]))
        return images

    return "", 400

In [4]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/May/2024 10:33:52] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [13/May/2024 10:33:52] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/May/2024 10:33:57] "GET /images HTTP/1.1" 200 -


None


127.0.0.1 - - [13/May/2024 10:34:19] "GET /images?latitude=32.323432 HTTP/1.1" 200 -


32.323432
